In [1]:
# this involves standardization and normalization and is important as some models are sensitive to the feature scales

'''
for the small datasets the numbers with very high fluctuations can distort the model so better to do scaling
standard is used when the datas are normally distibuted and bell shaped
minmax is rare and for some specific requirements only
'''
import pandas as pd
from sklearn.preprocessing import StandardScaler ,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso # this beacause our dataset is smaller and it helps prevent overfitting
from sklearn.compose import ColumnTransformer # using this to broaden my knowledge


In [8]:
df= pd.read_csv("cleaned_data.csv")

# separate target and features as scaling is done in features
x=df[['TV','Radio','Newspaper']] # for selecting more columns we have to use double squared
y=df['Sales']

# for feature scaling we have to split the datasets into train and test to prevent the data leakage and donot do cheating
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.3, random_state=42)


In [10]:
# standard scaler using columntransformer

# this is a object and blueprint of what actually has to be done (no caluculations)
scaler = ColumnTransformer(
    transformers=[ 
        # tuple used as we might apply different methods i.e. scaling or encoding according to the columns
        ('scaled', StandardScaler(),['TV','Radio','Newspaper'])

    ],
    remainder='passthrough' # this keeps the unused columns as it is without removing
)

# now scaling train and test data which will be numpy array

# fit transform done so scaler lerans about the statisitcs of training data and tranforms / scales them
x_train_Scaled= scaler.fit_transform(x_train) 

# only transforming test data as fiting will leak the datas because we are predicting the test data and doing fiting will help model to see data previously.
x_test_Scaled= scaler.transform(x_test) 

# converting these arrays to the dataframe i.e. creates new dataframe
x_train_Scaled = pd.DataFrame(x_train_Scaled,columns=['TV','Radio','Newspaper'])
x_test_Scaled=pd.DataFrame(x_test_Scaled,columns=['TV','Radio','Newspaper'])





In [11]:
# apply minmax scaler to compare
minmax=MinMaxScaler()

x_train_mm=minmax.fit_transform(x_train)
x_test_mm = minmax.transform(x_test)

# converting the numpy arrays from fit transform to datframe
x_train_df = pd.DataFrame(x_train_mm, columns=x.columns) 
# this x.columns sues the all columns original of the splited x feature i.e. the easiest way to do



In [14]:
# now comparing the two scaling models
print ('standard scaler summary')
print(x_test_Scaled.describe())

print('\n')

print(' summary statistics minmax')
print(x_train_df.describe())



standard scaler summary
              TV      Radio  Newspaper
count  60.000000  60.000000  60.000000
mean   -0.372680  -0.059658  -0.030828
std     1.041877   1.058536   1.045048
min    -1.806099  -1.558298  -2.798888
25%    -1.241205  -1.080009  -0.619956
50%    -0.551310  -0.042857   0.085558
75%     0.511195   0.812578   0.703035
max     1.632910   1.774300   1.874676


 summary statistics minmax
               TV       Radio   Newspaper
count  140.000000  140.000000  140.000000
mean     0.526511    0.474294    0.617848
std      0.283733    0.295072    0.221540
min      0.000000    0.000000    0.000000
25%      0.312225    0.216734    0.464561
50%      0.559689    0.462702    0.651114
75%      0.752621    0.742440    0.793081
max      1.000000    1.000000    1.000000
